In [2]:
import requests
import wikipedia
import string
from wikiparse import WikiParser
%load_ext autoreload
%autoreload 2

In [3]:
S = requests.Session()

URL = "https://en.wikipedia.org/w/api.php"

TITLE = "Category:DC Comics supervillains"

PARAMS = {
    'action': "query",
    'list': 'categorymembers',
    'cmtitle': TITLE,
    'cmlimit': '100',
    'format': "json",
}

R = S.get(url=URL, params=PARAMS)
DATA = R.json()
ids = [item['title'] for ind,item in enumerate(DATA['query']['categorymembers'])]
ids[:20]

['Kyle Abbot',
 'Abra Kadabra (comics)',
 'Ace of Spades (comics)',
 "Whisper A'Daire",
 'Aegeus (comics)',
 'Agamemno',
 'Agent Axis (DC Comics)',
 'Agent Liberty',
 'Larfleeze',
 "Ra's al Ghul",
 'Talia al Ghul',
 'Beth Kane',
 'Alpha (DC Comics)',
 'Amazo',
 'Amygdala (comics)',
 'Anarky',
 'Hawkman (Fel Andar)',
 'Angle Man',
 'Animal-Vegetable-Mineral Man',
 'Anti-Monitor']

In [5]:
len(ids)

100

In [4]:
bios = []
miss = 0

for id in ids:
    try:
        hero = wikipedia.page(id)
        bio = get_bio(hero)
        if bio != -1:
            bios.append(bio)
    except wikipedia.exceptions.DisambiguationError:
        miss += 1

In [4]:
page = wikipedia.page('abigail Brand')
w = WikiParser()
# tests = w.get_all_bios(ids,'villain')
# tests[:5]

In [91]:
fulltext = c.content

In [19]:
#S = requests.Session()
import datetime
import time

now = datetime.datetime.now()

wp = WikiParser()
cat = "Category:Marvel Comics superheroes"
titles = wp.get_category(cat)

h_bios = []
old_bios = []

for _ in range(15):
    print('page: ' + str(_) + ' parsing...', end=" ")
    new_bios = wp.get_all_bios(titles,'hero')
    if new_bios == old_bios:
        break
    h_bios += new_bios
    if wp.cmc != -1:
        titles = wp.continue_category(cat)
    old_bios = new_bios

later = datetime.datetime.now()
elapsed = later-now
print("Time: ", elapsed) 
heroes = h_bios
print(len(h_bios))

page: 0 parsing... . . . . . . . . . .  
page: 1 parsing... . . . . . . . . . .  
page: 2 parsing... . . . . . . . . . .  
page: 3 parsing... . . . . . . . . . .  
page: 4 parsing... . . . . . . . . . .  
page: 5 parsing... . . . . . . . . . .  
page: 6 parsing... . . . . . . . . . .  
page: 7 parsing... . . . . . . . . . .  
page: 8 parsing... . . . . . . . . .  
page: 9 parsing... . . . . . . . . .  
Time:  0:09:15.125692
382


In [20]:
len(heroes)

380

In [16]:
#S = requests.Session()
import datetime
import time

now = datetime.datetime.now()

wp_v = WikiParser()
cat = "Category:Marvel Comics supervillains"
titles = wp_v.get_category(cat)

v_bios = []
old_bios = []

for _ in range(12):
    print('page: ' + str(_) + ' parsing...', end=" ")
    new_bios = wp_v.get_all_bios(titles,'villain')
    if new_bios == old_bios:
        break
    v_bios += new_bios
    if wp_v.cmc != -1:
        titles = wp_v.continue_category(cat)
    old_bios = new_bios
        
villains = v_bios
later = datetime.datetime.now()
elapsed = later-now
print("Time: ", elapsed)    
print(len(v_bios))

page: 0 parsing... . . . . . . . . . .  
page: 1 parsing... . . . . . . . . . .  
page: 2 parsing... . . . . . . . . . .  
page: 3 parsing... . . . . . . . . . .  
page: 4 parsing... . . . . . . . . . .  
page: 5 parsing... . . . . . . . . . .  
page: 6 parsing... . . . . . . . . . .  
page: 7 parsing... . . . . . . . . . .  
page: 8 parsing... . . . . . . . . . .  
page: 9 parsing... . . . . . . . . . .  
page: 10 parsing... . . . . . . . . . .  
page: 11 parsing... . .  
Time:  0:17:22.420288
382


In [17]:
v_bios = bios
villains = v_bios
len(villains)

382

In [21]:
all_marvel = villains + heroes

In [22]:
len(all_marvel)

762

In [81]:
len(wp_v.ambiguation)

20

In [25]:
import pandas as pd

marvel_df = pd.DataFrame.from_dict(all_marvel)
marvel_df.head()
marvel_df.to_json('marvel_bios.json')

In [78]:
marvel_df[~marvel_df['bio'].isna()].shape

(793, 2)

In [5]:
wp = WikiParser()
cat = "Category:DC Comics superheroes"
titles = wp.get_category(cat)

bios = []
old_bios = []

for _ in range(10):
    print('page: ' + str(_) + ' parsing...', end=" ")
    new_bios = wp.get_all_bios(titles,'hero')
    if new_bios == old_bios:
        break
    bios += new_bios
    if wp.cmc != -1:
        titles = wp.continue_category(cat)
    old_bios = new_bios
    
print(len(bios))
dc_heroes = bios

page: 0 parsing... . . . . . . . . . .  
page: 1 parsing... . . . . . . . . . .  
page: 2 parsing... . . . . . . . . . .  
page: 3 parsing... . . . . . . . . . .  
page: 4 parsing... . . . . . . . . . .  
page: 5 parsing... . . . . . . . . . .  
page: 6 parsing... . . . . . . . . . .  
page: 7 parsing... . . . . . . . . . .  
page: 8 parsing... . . . . . . .  
page: 9 parsing... . . . . . . .  


In [8]:
len(dc_heroes)
len(wp.ambiguation)

6

In [9]:
wp_v = WikiParser()
cat = "Category:DC Comics supervillains"
titles = wp_v.get_category(cat)

bios = []

for _ in range(8):
    print('page: ' + str(_) + ' parsing...', end=" ")
    new_bios = wp_v.get_all_bios(titles,'villain')
    bios += new_bios
    if wp_v.cmc != -1:
        titles = wp_v.continue_category(cat)
    
print(len(bios))
dc_villains = bios

page: 0 parsing... . . . . . . . . . .  
page: 1 parsing... . . . . . . . . . .  
page: 2 parsing... . . . . . . . . . .  
page: 3 parsing... . . . . . . . . . .  
page: 4 parsing... . . . . . . . . . .  
page: 5 parsing... . . . . . . . . . .  
page: 6 parsing... . . . . . . . . . .  
page: 7 parsing... . . . . . .  
174
